<a href="https://colab.research.google.com/github/dm-fedorov/pandas_basic/blob/master/%D1%83%D0%BF%D1%80%D0%B0%D0%B6%D0%BD%D0%B5%D0%BD%D0%B8%D1%8F/5.%20%D0%9E%D0%B1%D1%8A%D0%B5%D0%B4%D0%B8%D0%BD%D0%B5%D0%BD%D0%B8%D0%B5%20%D1%82%D0%B0%D0%B1%D0%BB%D0%B8%D1%86.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory" target="_blank"></a>

Будем работать с [логами рекомендательной системы фильмов](https://grouplens.org/datasets/movielens/). Основой данных будут две таблицы. Первая — это данные о выставленных оценках фильмов:

In [1]:
import pandas as pd

url = "https://raw.githubusercontent.com/dm-fedorov/pandas_basic/master/data/ratings.csv"

ratings = pd.read_csv(url)
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


Состав столбцов:

- `userId` — идентификатор пользователя, который поставил фильму оценку
- `movieId` — идентификатор фильма
- `rating` — выставленная оценка
- `timestamp` — время (в формате unix time), когда была выставлена оценка

Вторая таблица — расшифровка идентификаторов фильмов:

In [2]:
url = "https://raw.githubusercontent.com/dm-fedorov/pandas_basic/master/data/movies.csv"

movies = pd.read_csv(url)
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


Состав столбцов:

- `movieId` — идентификатор фильма
- `title` — название фильма
- `genres` — список жанров, к которым относится фильм

У датафреймов `ratings` и `movies` есть общий столбец `movieId`. Значит, мы можем объединить эти датафреймы в одну таблицу. Используем метод [`merge`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html):

In [3]:
joined = ratings.merge(movies, on='movieId', how='left')
joined.head() 

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


Схематично: 

```Python
joined = left_df.merge(right_df, on='', how=''), где
```

- `left_df` / `right_df` — датафреймы, которые мы объединяем. К "правому" датафрейму присоединяем "левый" (в нашем примере "левый" датафрейм — `ratings`, "правый" — `movies`). 
- `how` — параметр объединения записей. Он может иметь четыре значения: `left`, `right`, `inner` и `outer`. При значении `left` берем все записи (`movieId`) из "левого" датафрейма (`ratings`) и ищем их соответствия в "правом" (`movies`). В итоговом датафрейме останутся только те значения, которым были найдены соответствия, то есть только значения из `ratings`. Аналогично при параметре `right` остаются только значения из "правого" датафрейма. Если совпадений между таблицами нет, то ставим нулевое значение. Значение `inner` оставляет только те записи (`movieId`), которые есть в обоих датафреймах, `outer` объединяет все варианты `movieId` в обоих датафреймах. 
- `on` определяет, по какому столбцу происходит объединение. Для объединения по нескольким столбцам используйте `on = ['col1', 'col2']` или `left_on` и `right_on`.


<img src="https://raw.githubusercontent.com/dm-fedorov/pandas_basic/master/%D1%81%D0%BB%D0%BE%D0%B6%D0%BD%D1%8B%D0%B5%20%D1%82%D0%B5%D0%BC%D1%8B%20pandas/pic/merge.png">



После объединения датафреймов лучше проверять, что не возникло дубликатов. 

Убедимся в том, что число строк объединенного датафрейма совпадает с исходным:

In [4]:
len(ratings) == len(joined)

True

Получаем значение True — значит, число строк совпадает.

Объединение датафреймов с помощью метода `merge` имеет особенности, аналогичные [`SQL JOIN`](http://www.skillz.ru/dev/php/article-Obyasnenie_SQL_obedinenii_JOIN_INNER_OUTER.html). Если точнее, есть ситуации, которые приводят к дублированию строк в конечном результате. Разберем эти ситуации более подробно на примере небольших таблиц:

In [5]:
import pandas as pd

url = "https://raw.githubusercontent.com/dm-fedorov/pandas_basic/master/data/ratings_example.txt"

ratings = pd.read_csv(url, sep='\t')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144


In [6]:
url = "https://raw.githubusercontent.com/dm-fedorov/pandas_basic/master/data/movies_example.txt"

movies = pd.read_csv(url, sep='\t')
movies.head()

,movieId,title,genres
0,31,Dangerous Minds (1995),Drama
1,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
2,31,Dangerous Minds (1995),Drama


**Дубликаты строк**

Итак, в датафрейме `movies` есть две строки с одним `movieId`. То есть теперь для таблицы `ratings` нет однозначного соответствия, с какой строкой она может объединиться с таблицей `movies`. В итоге строка с `movieId = 31` будет дублирована:

In [7]:
ratings.merge(movies, how='left', on='movieId')

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,31,2.5,1260759144,Dangerous Minds (1995),Drama


Если вы хотите избежать подобной ситуации, необходимо удалить дубликаты из таблицы `movies`. Для этого подходит метод `drop_duplicates`. В параметре `subset` указываем один или несколько столбцов, по комбинации которых хотим удалить дубликаты.

С помощью параметра `keep` указываем, какой из встречающихся дубликатов оставить (например, первый или последний). Параметр `inplace` указывает, что изменения нужно сохранить в датафрейме, к которому применяется метод (в нашем случае — в датафрейме `movies`):

In [8]:
movies.drop_duplicates(subset='movieId', keep='first', inplace=True)
movies.head()

,movieId,title,genres
0,31,Dangerous Minds (1995),Drama
1,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller


Теперь объединение таблиц будет корректным:

In [9]:
ratings.merge(movies, how='left', on='movieId')

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama


### Задание

Рассмотрим данные о новых поступлениях в интернет-магазин. 

В словаре `items_dict` содержится информация о наличии товара на складе:

In [10]:
items_dict = {
    'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 100132, 312394], 
    'vendor': ['Samsung', 'LG', 'Apple', 'Apple', 'LG', 'Apple', 'Samsung', 'Samsung', 'LG', 'ZTE'],
    'stock_count': [54, 33, 122, 18, 102, 43, 77, 143, 60, 19]
}

А в словаре `purchase_log` — данные о покупках товаров:

In [11]:
purchase_log = {
    'purchase_id': [101, 101, 101, 112, 121, 145, 145, 145, 145, 221],
    'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 103845, 100132], 
    'price': [13900, 5330, 38200, 49990, 9890, 33000, 67500, 34500, 89900, 11400]
}

Состав столбцов:

- `item_id` — идентификатор модели (по этому столбцу будем объединять датафреймы)
- `vendor` — производитель модели
- `stock_count` — имеющееся на складе количество данных моделей (в штуках)
- `purchase_id` — идентификатор покупки
- `price` — стоимость модели в покупке

Переведем сначала эти словари в датафреймы:

In [12]:
items_df = pd.DataFrame(items_dict)
purchase_df = pd.DataFrame(purchase_log)

### Задание 1

Объедините получившиеся датафреймы по столбцу `item_id` с типом `outer`.

Определите, модель с каким `item_id` есть в статистике продаж `purchase_df`, но не учтена на складе (подсказка: подумайте, какой датафрейм должен быть "левым", а какой "правым", чтобы получить необходимые данные). Ответ в виде целого числа.

### Задание 2

Решите обратную задачу: модель с каким `item_id` есть на складе, но не имела ни одной продажи? Ответ в виде целого числа.

### Задание 3

Сформируйте датафрейм `merged`, в котором в результате объединения `purchase_df` и `items_df` останутся модели, которые учтены на складе и имели продажи. Сколько всего таких моделей?

### Задание 4

Посчитайте объем выручки для каждой модели, которую можно получить, распродав все остатки на складе. Модель с каким `item_id` имеет максимальное значение выручки после распродажи остатков? Ответ в виде целого числа.

Примечание: перемножение столбцов датафрейма можно производить разными способами, но самый простой - перемножение "в лоб" вида `df['col1'] = df['col2'] * df['col3']`. Для присоединения новых данных к датафрейму тоже можно использовать различные методы, включая функцию `.append()`, которая позволяет присоединять к датафрейму другой датафрейм, серии или словари.

### Задание 5

Посчитайте итоговую выручку из прошлого задания по всем моделям. Ответ в виде целого числа.